In [2]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, roc_auc_score

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

## Loading Training/Testing/Validation Data

In [3]:
dataset = "Wafer"
model_name = "ladder"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = np.reshape(x_train_load['arr_0'], [x_train_load['arr_0'].shape[0], x_train_load['arr_0'].shape[1], 1])
x_test = np.reshape(x_test_load['arr_0'], [x_test_load['arr_0'].shape[0], x_test_load['arr_0'].shape[1], 1])

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']

y_all = np.concatenate((y_train, y_test), axis = 0)
y_all = np.asarray(y_all, dtype = np.uint64)

n_validation = int(0.1*n_instances)
print(f"Validation Instances: {n_validation}")

ind_validation = random.sample(range(0, n_instances), n_validation)
x_valid = x_all[ind_validation, :, :]
y_valid = y_all[ind_validation]

x_all = np.delete(x_all, ind_validation, axis = 0)
y_all = np.delete(y_all, ind_validation, axis = 0)

print(f"x_all: {x_all.shape} - y_all: {y_all.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_all: (6448, 152, 1) - y_all: (6448,)
x_valid: (716, 152, 1) - y_valid: (716,)


In [4]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_all.shape[1]
n_features = x_all.shape[-1]

model = Sequential()

model.add(RNN(LadderCell(units = 212,
                             max_delay = length,
                             input_dims = n_features), 
              input_shape = (length, n_features),
             return_sequences = False))
model.add(Dense(to_categorical(y_all).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 212)               101522    
_________________________________________________________________
dense (Dense)                (None, 2)                 426       
Total params: 101,948
Trainable params: 78,231
Non-trainable params: 23,717
_________________________________________________________________


In [5]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

kf = KFold(n_splits = 5)
sdk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
df_tpr = pd.DataFrame(columns = [1, 2, 3, 4, 5])
df_fpr = pd.DataFrame(columns = [1, 2, 3, 4, 5])

n_fold = 1

acc_per_fold = []
epoch_per_fold = []
loss_per_fold = []
rec_per_fold = []
prec_per_fold = []
f1_per_fold = []
auroc_per_fold = []
fpr_per_fold = []
tpr_per_fold = []

for train, test in sdk.split(x_all, y_all):
    
    file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-{model_name}_{n_fold}.hdf5'))

    callbacks = [
        ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
        EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min')]

    t = time.time()
    
    print(f"\nFold: {n_fold}\n")
    result = model.fit(x_all[train], 
                       to_categorical(y_all[train]),
                       epochs = 200, 
                       batch_size = 100, 
                       validation_data = (x_valid, to_categorical(y_valid)), 
                       callbacks = callbacks)

    print(f"Training time: {time.time() - t} s")

    df_results = pd.DataFrame(result.history)
    df_results.to_csv(os.path.abspath(os.path.join('models', dataset, f'{model_name}_results_{n_fold}.csv')))
    
    model.load_weights(file_path)
    scores = model.evaluate(x_all[test], to_categorical(y_all[test]))
    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1])
    epoch_per_fold.append(np.argmin(result.history['val_loss']))
    
    # Computing predictions
    y_pred_test = np.argmax(model.predict(x_all[test]), axis = 1)
    
    # Getting performance scores per fold
    f1_per_fold.append(f1_score(y_all[test], y_pred_test, average = 'macro'))
    rec_per_fold.append(recall_score(y_all[test], y_pred_test, average = 'macro'))
    prec_per_fold.append(precision_score(y_all[test], y_pred_test, average = 'macro'))
    auroc_per_fold.append(roc_auc_score(y_all[test], y_pred_test, average = 'macro'))
    
    # Getting ROC curve for this binary classification task
    
    fpr, tpr, _ = roc_curve(y_all[test], y_pred_test)
    
    df_fpr[n_fold] = fpr
    df_tpr[n_fold] = tpr
        
    n_fold += 1

df_scores = pd.DataFrame(columns = ['F1', 'Loss', 'Accuracy', 'Precision', 'Recall', 'AUROC'])
df_scores['F1'] = f1_per_fold
df_scores['Loss'] = loss_per_fold
df_scores['Accuracy'] = acc_per_fold
df_scores['Precision'] = prec_per_fold
df_scores['Recall'] = rec_per_fold
df_scores['AUROC'] = auroc_per_fold

df_scores.to_csv(os.path.abspath(os.path.join('models', dataset, f'{model_name}_results_k-Folds.csv')))
df_tpr.to_csv(os.path.abspath(os.path.join('models', dataset, f'{model_name}_results_k-Folds_tpr.csv')))
df_fpr.to_csv(os.path.abspath(os.path.join('models', dataset, f'{model_name}_results_k-Folds_fpr.csv')))

Fold: 1

Train on 5158 samples, validate on 716 samples
Epoch 1/200
5100/5158 [============================>.] - ETA: 0s - loss: 0.3138 - acc: 0.8725
Epoch 00001: val_loss improved from inf to 0.13289, saving model to /data/home/dorads/Documents/GitHub_Repos/01_Maintained/NeuralODE/models/Wafer/Wafer-ladder_1.hdf5
5158/5158 [==============================] - 5s 915us/sample - loss: 0.3123 - acc: 0.8732 - val_loss: 0.1329 - val_acc: 0.9567
Epoch 2/200
5100/5158 [============================>.] - ETA: 0s - loss: 0.0980 - acc: 0.9692
Epoch 00002: val_loss improved from 0.13289 to 0.07334, saving model to /data/home/dorads/Documents/GitHub_Repos/01_Maintained/NeuralODE/models/Wafer/Wafer-ladder_1.hdf5
5158/5158 [==============================] - 4s 797us/sample - loss: 0.0983 - acc: 0.9688 - val_loss: 0.0733 - val_acc: 0.9791
Epoch 3/200
5100/5158 [============================>.] - ETA: 0s - loss: 0.0553 - acc: 0.9833
Epoch 00003: val_loss improved from 0.07334 to 0.05466, saving model to 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/home/dorads/miniconda3/envs/sd_dev_tf_1/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-4f1b1b46b2be>", line 37, in <module>
    callbacks = callbacks)
  File "/data/home/dorads/miniconda3/envs/sd_dev_tf_1/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 727, in fit
    use_multiprocessing=use_multiprocessing)
  File "/data/home/dorads/miniconda3/envs/sd_dev_tf_1/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 675, in fit
    steps_name='steps_per_epoch')
  File "/data/home/dorads/miniconda3/envs/sd_dev_tf_1/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 394, in model_iteration
    batch_outs = f(ins_batch)
  File "/data/home/dorads/miniconda3/envs/sd_dev_tf_1/lib/python3.7/site-packages/tensorflow_core/py

TypeError: object of type 'NoneType' has no len()